### IMPORTING LIBRARIES

In [24]:
import os
from datetime import datetime, timedelta, date

import pandas as pd
import pandas_gbq
import pyarrow
import geopandas as gpd
import folium
import yaml
import wget
import zipfile

from google.cloud import bigquery
from google.oauth2 import service_account


### SHP files download

In [8]:
voivodships = 'https://www.gis-support.pl/downloads/2022/wojewodztwa.zip?_ga=2.264760332.262421511.1673644978-156643027.1672997767'
registration_units = 'https://www.gis-support.pl/downloads/2022/jednostki_ewidencyjne.zip?_ga=2.195986735.262421511.1673644978-156643027.1672997767'

In [9]:
os.makedirs('./POLYGON_SHAPES', exist_ok=True)
wget.download(voivodships, './POLYGON_SHAPES/voivo.zip')
wget.download(registration_units, './POLYGON_SHAPES/cadastral.zip')

'./POLYGON_SHAPES/cadastral (1).zip'

In [10]:
with zipfile.ZipFile('./POLYGON_SHAPES/voivo.zip', 'r') as zip_ref:
    zip_ref.extractall('./POLYGON_SHAPES/voivo')
with zipfile.ZipFile('./POLYGON_SHAPES/cadastral.zip', 'r') as zip_ref:
    zip_ref.extractall('./POLYGON_SHAPES/cadastral')

### Map importing:

#### Registration units

In [25]:
map_cadastral = gpd.read_file('./POLYGON_SHAPES/cadastral/jednostki_ewidencyjne.shp')
map_cadastral = map_cadastral[['JPT_KOD_JE', 'geometry']]
map_cadastral.shape

(3163, 2)

#### Voivodships

In [12]:
map_voivo = gpd.read_file('./POLYGON_SHAPES/voivo/wojewodztwa.shp')

map_voivo = map_voivo[['JPT_KOD_JE', 'geometry']]

map_voivo['woj'] = ['śląskie','opolskie','świętokrzyskie',
       'pomorskie','podlaskie','zachodniopomorskie',
       'dolnośląskie','wielkopolskie','podkarpackie',
       'małopolskie','warmińsko-mazurskie','łódzkie',
       'mazowieckie','kujawsko-pomorskie','lubelskie','lubuskie']

WOJ = map_voivo[['JPT_KOD_JE','woj']]

map_voivo.shape

(16, 3)

### Connecting to BIGQUERY

#### Take variables from notebook.yaml file

In [13]:
with open('./configuration_files/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

PROJECT_ID = config['PROJECT_ID']
DATASET_NAME = config['DATASET_NAME']
TABLE_PERMISSIONS = config['TABLE_PERMISSIONS']
TABLE_AGGREGATES = config['TABLE_AGGREGATES']
JSON_KEY_BQ = config['JSON_KEY_BQ']
PERIOD = 'last_3_m' # 'last_2_m' 'last_1_m'

#### Or assign here

In [14]:
key_path = f"configuration_files/{JSON_KEY_BQ}"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

## NUMBER OF PERMITS FOR VOIVODESHIP LAST 3 MONTHS

In [15]:
QUERY = f""" 
SELECT 
    *
FROM 
    `{PROJECT_ID}.{DATASET_NAME}.{TABLE_AGGREGATES}` 
WHERE
    length(unit_id) = 2
AND
    STRING(injection_date) LIKE '{date.today().strftime("%Y-%m")}%'
;
"""

perm_count = pandas_gbq.read_gbq(QUERY,credentials=credentials, progress_bar_type=None)

perm_count = perm_count[[col for col in perm_count.columns if PERIOD in col]+['unit_id']]
perm_count = perm_count.groupby(['unit_id']).sum()
perm_count = perm_count.sum(axis=1)

perm_count = perm_count.reset_index().rename(columns={0:'count'})

### Changed do markdown
mapa = folium.Map([52, 19], zoom_start=6)
folium.Choropleth(geo_data=map_voivo, # df with geographical data of areas
                  data=perm_count, # df with the data to show
                  columns=['unit_id', 'count'], # key column, value column
                  key_on='feature.properties.JPT_KOD_JE', # df key 
                  fill_color='YlOrRd', 
                  nan_fill_color="White",
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name="Number of permissions").add_to(mapa)
 
# saving created map into HTML file
os.makedirs('./OPENSTREETMAP_HTML', exist_ok=True)
mapa.save(outfile = f'OPENSTREETMAP_HTML/voivodships_{PERIOD}.html')
 
# Show map
mapa

![img](https://i.ibb.co/rmfxXz7/img-voivo.png)

## NUMBER OF PERMITS FOR REGISTRATION UNITS LAST 3 MONTHS

In [18]:
QUERY = f""" 
SELECT 
    *
FROM 
    `{PROJECT_ID}.{DATASET_NAME}.{TABLE_AGGREGATES}` 
WHERE
    length(unit_id) = 8
AND
    STRING(injection_date) LIKE '{date.today().strftime("%Y-%m")}%'
;
"""

perm_count = pandas_gbq.read_gbq(QUERY,credentials=credentials, progress_bar_type=None)

perm_count = perm_count[[col for col in perm_count.columns if PERIOD in col]+['unit_id']]
perm_count = perm_count.groupby(['unit_id']).sum()
perm_count = perm_count.sum(axis=1)

perm_count = perm_count.reset_index().rename(columns={0:'count'})

#### BEFORE MAP GENERATION SIMPLIFY SHAPES TO USE LESS MEMORY

In [19]:
map_cadastral.geometry = map_cadastral.geometry.simplify(1) # smaller value = more accurate

### Because of large file changed to markdown

mapa = folium.Map([52, 19], zoom_start=6)
folium.Choropleth(geo_data=map_cadastral, # df with geographical data of areas
                  data=perm_count, # df with the data to show
                  columns=['unit_id', 'count'], # key column, value column
                  key_on='feature.properties.JPT_KOD_JE', # df key  
                  fill_color='YlOrRd', 
                  nan_fill_color="White",
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name="Number of permissions").add_to(mapa)
 
# saving created map into HTML file
mapa.save(outfile = f'OPENSTREETMAP_HTML/cadastral_units{PERIOD}.html')
 
# Show map
mapa

![IMG](https://i.ibb.co/fnWT8hs/img-cadastral.png)